In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple
import os

In [2]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder {folder_path} created successfully.")
    else:
        print(f"Folder {folder_path} already exists.")


create_folder_if_not_exists("plots")
create_folder_if_not_exists("errors")

Folder plots already exists.
Folder errors already exists.


In [3]:
def get_A1(n, variable_type):
    A_ = np.zeros((n, n), dtype=variable_type)
    N = np.zeros(n, dtype=variable_type)
    for i in range(n):
        tmp = variable_type(i)
        N[i] = tmp
    for i in N:
        for j in N:
            index_i = int(i)
            index_j = int(j)
            if i - variable_type(1) < 0.001:
                A_[index_i][index_j] = variable_type(1)
            else:
                one = variable_type(1)
                A_[index_i][index_j] = one / ((i + one) + (j + one) - one)
    return A_


get_A1(5, np.float64)


array([[1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [0.33333333, 0.25      , 0.2       , 0.16666667, 0.14285714],
       [0.25      , 0.2       , 0.16666667, 0.14285714, 0.125     ],
       [0.2       , 0.16666667, 0.14285714, 0.125     , 0.11111111]])

In [4]:
def get_A2(n, variable_type):
    A_ = np.zeros((n, n), dtype=variable_type)
    N = np.zeros(n, dtype=variable_type)
    one = variable_type(1)
    two = variable_type(2)
    for i in range(n):
        tmp = variable_type(i)
        N[i] = tmp
    for i in N:
        for j in N:
            index_i = int(i)
            index_j = int(j)
            if j >= i:
                A_[index_i][index_j] = (two * (i + one)) / (j + one)
            else:
                A_[index_i][index_j] = (two * (j + one)) / (i + one)
    return A_
get_A2(5, np.float64)

array([[2.        , 1.        , 0.66666667, 0.5       , 0.4       ],
       [1.        , 2.        , 1.33333333, 1.        , 0.8       ],
       [0.66666667, 1.33333333, 2.        , 1.5       , 1.2       ],
       [0.5       , 1.        , 1.5       , 2.        , 1.6       ],
       [0.4       , 0.8       , 1.2       , 1.6       , 2.        ]])

In [5]:
def get_random_sign():
    return np.random.choice([-1, 1])


def get_x(n, variable_type):
    x_ = np.zeros(n, dtype=variable_type)
    for i in range(n):
        x_[i] = variable_type(variable_type(get_random_sign()))
    return x_

In [6]:
import csv


def save_to_csv(filename, data):
    filename = "errors/" + filename + ".csv"
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

In [7]:
def get_b(A, x, n, variable_type):
    b = np.zeros(n, dtype=variable_type)
    for i in range(n):
        for j in range(n):
            b[i] += A[i][j] * x[j]
        print()
    return b

In [8]:
#write a function which is given a matrix A and a vector b and returns the solution vector x
def gauss(A, b, variable_type):
    n = len(A)
    x = np.zeros(n, dtype=variable_type)
    for i in range(n):
        for j in range(i + 1, n):
            if A[i][i] == variable_type(0):
                print("Division by 0")
                return
            m = A[j][i] / A[i][i]
            for k in range(i, n):
                A[j][k] -= m * A[i][k]
            b[j] -= m * b[i]
    print(A)
    print(b)
    for i in range(n - 1, -1, -1):
        if A[i][i] == variable_type(0):
            print("Division by 0")
            return
        x[i] = b[i] / A[i][i]
        for j in range(i - 1, -1, -1):
            b[j] -= A[j][i] * x[i]
    #print(b)
    return x

In [9]:
def thomas(A, b, variable_type):
    n = len(A)
    x = np.zeros(n, dtype=variable_type)
    betas = np.zeros(n, dtype=variable_type)
    betas[0] = - A[0][1] / A[0][0]
    gammas = np.zeros(n, dtype=variable_type)
    gammas[0] = b[0] / A[0][0]
    for i in range(1, n):
        if i == n - 1:
            betas[i] = 0
        else:
            betas[i] = - A[i][i + 1] / (A[i][i] + A[i][i - 1] * betas[i - 1])
        gammas[i] = (b[i] - A[i][i - 1] * gammas[i - 1]) / (A[i][i] + A[i][i - 1] * betas[i - 1])
    x[-1] = gammas[-1]
    for i in range(n - 2, -1, -1):
        x[i] = betas[i] * x[i + 1] + gammas[i]
    return x

**Wykonanie*

In [14]:
#write function generating random band matrix
def get_band_matrix(n, variable_type):
    A = np.zeros((n, n), dtype=variable_type)
    for i in range(n):
        for j in range(n):
            if i == j:
                A[i][j] = variable_type(get_random_sign())
            elif i == j + 1 or i == j - 1:
                A[i][j] = variable_type(get_random_sign())
    return A

#wirte a function which returns vector b of size n of random elements of type variable_type
def get_random_b(n, variable_type):
    b = np.zeros(n, dtype=variable_type)
    for i in range(n):
        b[i] = variable_type(get_random_sign())
    return b


#write a funciont which test correctness of thomas algorithm by comparing it with np.linalg.solve
def test_thomas(n, variable_type):
    A = get_band_matrix(n, variable_type)
    b = get_random_b(n, variable_type)
    x = thomas(A, b, variable_type)
    x_np = np.linalg.solve(A, b)
    print(x)
    print(x_np)
    print(np.allclose(x, x_np))
    return np.allclose(x, x_np)

[-9.71906220e-01 -1.97190622e+00 -3.94381244e+00  2.97190622e+00
 -2.80937805e-02  1.94381244e+00  9.71906220e-01  1.91571866e+00
 -3.88762488e+00 -6.80334354e+00 -1.91571866e+00  5.88762488e+00
  2.97190622e+00 -7.85953110e+00  3.88762488e+00 -4.97190622e+00
  2.08428134e+00  1.88762488e+00  1.19665646e+00  3.09031585e-01
 -1.88762488e+00  2.57859329e+00  3.46621817e+00  7.04481146e+00
  4.57859329e+00  1.06234048e+01  5.04481146e+00  4.57859329e+00
  1.46621817e+00 -2.11237512e+00 -1.64615695e+00 -5.33781829e-01
  1.17993878e+00 -7.13720608e-01  1.46621817e+00  2.47502437e-01
 -2.18715734e-01 -1.46621817e+00  2.47502437e-01  2.18715734e-01
  9.71213298e-01  1.89929032e-01  2.16114233e+00 -1.35107136e+00
  1.89929032e-01  5.41000394e-01  2.69070574e-01  1.27192982e+00
  2.85924578e-03 -2.74789066e-01 -7.22351689e-01 -5.52437377e-01
  1.16991431e+00  7.22351689e-01  8.92266000e-01  8.30085688e-01
  1.06218031e+00  7.67905377e-01 -7.05725065e-01 -4.73630441e-01
 -1.79355506e-01 -1.65298

True